In [1]:
import pandas as pd
import requests

In [7]:
def download_carga_horaria(ano_inicio: int, ano_fim: int):
    """ Função para fazer download dos dados de carga elétrica por subsistema no período de referência em base diária."""

    #url = "https://ons-dl-prod-opendata.s3.amazonaws.com/dataset/carga_energia_di/CARGA_ENERGIA_{}.csv"
    url = "https://ons-dl-prod-opendata.s3.amazonaws.com/dataset/curva-carga-ho/CURVA_CARGA_{}.csv"
    # verificar se anos inicial e final estão disponíveis
    get0 = requests.get(url.format(ano_inicio)).status_code # verify = False (autenticação)
    getn = requests.get(url.format(ano_fim)).status_code 
    if (get0 == 200) and (getn == 200): # 200: página (ano) disponível

        # concatenar arquivos de cada ano em um único dataframe
        df = pd.DataFrame()
        for ano in range(ano_inicio, ano_fim + 1):
            df2 = pd.read_csv(url.format(ano), sep = ";")
            df = pd.concat([df, df2])
        df.columns = ["id_reg", "desc_reg", "date", "load_mwmed"]
        df.loc[:, "date"] = pd.to_datetime(df.loc[:, "date"], format = '%Y-%m-%d %H:%M:%S')
        df.sort_values(by = "date", inplace = True)
        return df
    
    else:
       print("Ano não disponível.")


def load_data():
    """
    Função para ler e transformar os dados já presentes no diretório especificado
    """
    path = "hourly_load.csv"
    df_load = pd.read_csv(path, parse_dates = ["date"])
    df_load2 = df_load[df_load["id_reg"] == "S"]           # região sul
    df_load3 = df_load2[df_load2["date"] < '2022-06-01']  # data de corte
    df_load4 = df_load3[["date", "load_mwmed"]].set_index("date")
    return df_load4

In [12]:
# df = download_carga_horaria(2000, 2022)
# df.to_csv("hourly_load.csv")

In [9]:
df = load_data()
df.tail()

,load_mwmed
date,
2022-05-31 19:00:00,15044.825
2022-05-31 20:00:00,14277.058
2022-05-31 21:00:00,13664.627
2022-05-31 22:00:00,12612.263
2022-05-31 23:00:00,11290.224
